In [ ]:
"""
# 1.使用 ollama 的 openhermes 模型
# 2.使用 phidata 的命令行交互模式 assistant.cli_app()
# 3.可以調用 DuckDuckGo 搜尋網路資源
# 4.windows 環境偶而會發生ollama 模型載入失敗的問題，此時需開啟 powershell 視窗輸入 wsl --shutdown 關閉 wsl 環境後重開 ubuntu 環境
# 案例：
# Summarize the top stories on hackernews and reply in traditional chinese.
# Today's Top 10 trending Repositories on GitHub? Introduce them using traditional chinese.
# 爬取 GitHub Trending 頁面,以繁體中文解說前5的項目
# 爬取 GitHub Trending 頁面,以繁體中文解說前5的項目,請使用我已經設定在系統環境變數 GITHUB_API_KEY 來爬取 
# Summarize the top 3 stories on hackernews and translate them into traditional chinese.
# Summarize the top 3 stories on hackernews and reply in traditional chinese. 
# Whats happening in Taiwan today? Summarize top 3 stories with sources and translate to traditional chinese.
# Whats happening in Taiwan today? Summarize top 3 stories with sources and reply in traditional chinese. 
# Write a python script to plot a sine wave and save it to disc as a png file sine_wave.png
# 搜尋 arxiv 有關GPT-4最新的3篇論文,請以繁體中文markdown格式存檔到 arxiv.md  
# 搜尋網路並推薦台中2日遊的景點及行程規劃
"""

In [ ]:
import textwrap
from IPython.display import HTML, display, Markdown

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
# 匯入依賴庫（套件）
from phi.assistant import Assistant
from phi.assistant.python import PythonAssistant
from phi.llm.ollama import Ollama
from phi.llm.openai.like import OpenAILike
from pydantic import BaseModel, Field

In [ ]:
from phi.assistant import Assistant
from phi.llm.openai import OpenAIChat

assistant = Assistant(
    llm=OpenAIChat(model="gpt-3.5-turbo"),
)
# assistant.cli_app(markdown=True)

In [ ]:
# 匯入 tools(部分)
from phi.tools.arxiv import ArxivTools
from phi.tools.duckduckgo import DuckDuckGo
from phi.tools.email import EmailTools
from phi.tools.file import FileTools
# from phi.tools.google import GoogleTools # 官方尚未完成
from phi.tools.python import PythonTools
from phi.tools.shell import ShellTools
from phi.tools.wikipedia import WikipediaTools
from phi.tools.website import WebsiteTools

In [ ]:
# 自訂函數當成工具
import httpx, json
def get_top_hackernews_stories(num_stories: int = 10) -> str:
    """Use this function to get top stories from Hacker News.

    Args:
        num_stories (int): Number of stories to return. Defaults to 10.

    Returns:
        str: JSON string of top stories.
    """

    # Fetch top story IDs
    response = httpx.get('https://hacker-news.firebaseio.com/v0/topstories.json')
    story_ids = response.json()

    # Fetch story details
    stories = []
    for story_id in story_ids[:num_stories]:
        story_response = httpx.get(f'https://hacker-news.firebaseio.com/v0/item/{story_id}.json')
        story = story_response.json()
        if "text" in story:
            story.pop("text", None)
        stories.append(story)
    return json.dumps(stories)

In [ ]:
assistant = PythonAssistant(
    # 沒有支援 gemini
	# llm=Ollama(model="codellama"),            # 可以跑出結果，但結果錯誤 4.917 
	# llm=Ollama(model="llama2"),               # 6.7、6.4 多次不同結果
	# llm=Ollama(model="llama2-uncensored"),    # 可以跑出結果，但結果錯誤 7.15 
	# llm=Ollama(model="llava"),                  # 可以跑出結果，但結果錯誤 6.5
	# llm=Ollama(model="mistral"),               # 任務失敗，沒跑出結果
	# llm=Ollama(model="neural-chat"),            # 可以跑出結果，但結果錯誤 6.9 
	# llm=Ollama(model="openchat"),               # The average rating of movies is not provided in the given data file.
	# llm=Ollama(model="openhermes"),           # 可以跑出結果，但結果錯誤 7.1
    # llm=Ollama(model="openhermes"),
	# llm=Ollama(model="qwen"),                 # 任務失敗，很久還跑出亂碼
	# llm=Ollama(model="starling-lm"),            # 可以跑出結果，但結果錯誤 7.3
	# 這些 ollama 的大模型都沒有成功調用 python 的 tools 來寫程式碼計算平均分數。失敗~
    
	# 如果使用 gpt-3.5-turbo 幾乎每次都成功, 函數調用也沒問題
    llm=OpenAIChat(model="gpt-3.5-turbo"),
	
	# llm=OpenAILike(base_url="http://localhost:1234/v1"),
	
	system_prompt="You ara a helpful assistant.",
	description="You can use provided tools to help users solve any problems.",
	use_tools=True,
	show_tool_calls=True,
	tools=[get_top_hackernews_stories, ArxivTools(), DuckDuckGo(), EmailTools(), FileTools(), PythonTools(), ShellTools(), WebsiteTools(), WikipediaTools()],  # GoogleTools(), 
	pip_install=True,
	save_and_run=True,
	run_code=True,
	debug_mode=True,
	markdown=True
	)


In [ ]:
# Summarize the top stories on hackernews today and translate to traditional chinese.
# Search the top 5 stories on hackernews today. Summarize them and give me the links.
# 搜尋 wikipedia,列出五月天歷年的專輯清單(無需其他說明),寫入檔案 五月天.txt
task="Summarize the top 5 stories on hackernews today. Reply in traditional chinese"

In [ ]:
assistant.print_response(task, markdown=True)
# assistant.cli_app(markdown=False)